In [1]:
import numpy as np
from qiskit import QuantumCircuit, Aer, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import DensityMatrix
from qiskit import assemble, transpile, execute

# Quantum Tomography

Base on: https://arxiv.org/pdf/1407.4759.pdf

In [2]:
I = np.array([[1,0],[0,1]])
px = np.array([[0,1],[1,0]])
py = np.array([[0,-1j],[1j,0]])
pz = np.array([[1,0],[0,-1]])

In [3]:

def Tomo1Qubit(qc):
    print('Real Density Matrix = \n {}'.format(DensityMatrix(qc)))
    Shots = 100000
    exp_values = ['x','y','z']

    exp = []
    for i in exp_values:
        weight_0 = 0
        weight_1 = 0
        
        qr = QuantumRegister(1, 'q')
        cr = ClassicalRegister(1,'c')
        circ = QuantumCircuit(qr, cr)
        circ.append(qc,[qr[0]])

        if i == 'x':
            circ.h(0)
        elif i == 'y':
            circ.sdg(0)
            circ.h(0)
        else:
            circ.i(0)

        circ.measure(qr[0],cr[0])

        Simulation = Aer.get_backend('qasm_simulator')
        Quantum_Transpiler = transpile(circ,Simulation)
        Quantum_Obj = assemble(Quantum_Transpiler, shots=Shots)
        Result = Simulation.run(Quantum_Obj).result()
        
        weight_0 = Result.get_counts().setdefault('0',0)
        weight_1 = Result.get_counts().setdefault('1',0)

        exp.append((weight_0-weight_1)/Shots)

    estimate_density = 0.5*(I + exp[0]*px + exp[1]*py + exp[2]*pz)

    print("Estimate density matrix = ")
    return estimate_density

In [4]:
qc = QuantumCircuit(1)
qc.h(0)
qc.t(0)
qc.rx(np.pi*0.3, 0)

Tomo1Qubit(qc)

Real Density Matrix = 
 DensityMatrix([[0.7860307 +0.j        , 0.35355339-0.20781347j],
               [0.35355339+0.20781347j, 0.2139693 +0.j        ]],
              dims=(2,))
Estimate density matrix = 


array([[0.78447+0.j     , 0.35219-0.20787j],
       [0.35219+0.20787j, 0.21553+0.j     ]])